<a href="https://colab.research.google.com/github/DOLARIK/The-Gaurdian/blob/master/The_Guardian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting Up

In [0]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil


In [0]:
import psutil
import humanize
import os
import GPUtil as GPU

In [0]:
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 


Gen RAM Free: 12.9 GB  | Proc size: 305.2 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [0]:
!pip install PyDrive

    100% |████████████████████████████████| 993kB 22.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import pickle
import hashlib
import gc
import time

In [0]:
# Simplifying the 'pickling' and 'driving' process as we'll be needing this a lot

def down_drive(id, name):
  # 'id' - id of the file you wish to download from drive
  # 'name' - name of that file
  download = drive.CreateFile({'id': id})
  download.GetContentFile(name)
  
def up_drive(title):
  # 'title' - title of the file you wish to upload over the drive
  file = drive.CreateFile({'title': title})
  file.SetContentFile(title)
  file.Upload()
  
def pickleIn(name):    
  # 'name' - name of the file we wish to load (should be a string, eg. 'train_batch.pickle')
  pickle_in = open(name,'rb')
  return pickle.load(pickle_in)

def pickleOut(name, to_be_stored):
  # 'name' - name of the file we wish to store (should be a string, eg. 'train_batch.pickle')
  # 'to_be_stored' - the variable of which we'll be making the "pickle" 
  pickle_out = open(name,'wb')
  pickle.dump(to_be_stored, pickle_out)
  pickle_out.close

# Starting the Real Work Up

## Importing Our Dataset

### Importing GloVe Embeddings


Downloading GloVe Embeddings

In [0]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2019-04-07 07:56:34--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2019-04-07 07:56:34--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Saving to: ‘glove.42B.300d.zip’

glove.42B.300d.zip  100%[===================>]   1.75G  52.8MB/s    in 31s     

2019-04-07 07:57:05 (57.8 MB/s) - ‘glove.42B.300d.zip’ saved [1877800501/1877800501]



In [0]:
!ls

glove.42B.300d.zip  sample_data


In [0]:
!unzip glove.42B.300d.zip

Archive:  glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [0]:
!ls -l glove.42B.300d.txt
!ls -l 1


-rw-rw-r-- 1 root root 5025028820 Oct 24  2015 glove.42B.300d.txt
ls: cannot access '1': No such file or directory


In [0]:
def show_size(size):
  size_dict = {1:'B',2:'KB',3:'MB',4:'GB',5:'TB',6:'PB'}
  count = 1

  while 1:
    size = size/1024

    if size < 1:
      if count <= 6:
        print(size_, size_dict[count])
      else:
        print(size_, size_dict[6])
      break

    size_ = size
    count += 1
  
glove_size = 5025028820 
show_size(glove_size)

4.67992277815938 GB


Seems like the Text file containing all GloVe Embeddings is too huge. 

This wouldn't have posed a very major issue had the script been meant to not run multiple times. If we invoke the script multiple times then it would take up a lot of RAM for every time it loads the text file.

So, instead of loading the complete file (at the time of run) we can do one of the following-
  
    1. Save a pickled embedding of each word separately - 
  
          a. This would take upto 5 Gigs of Space (almost the same as taken by OG txt file)
          b. For loading each word time taken would be approx ~ .0008 secs
          c. The average number of words in a novel is around 80,000. And, it would be safe to say that most webpages visited by a user have less words than that present in a novel.
          d. It takes approximately 1 minute to load the embeddings for all the words present in our website. (We can use this!! Double Check Required tho)
    
    2. Save differnt chucks of words based on starting letter (like words starting with 'a', 'b' etc..)
          
          a. This would divide the 4.67 Gigs into 27 different chunks (on an average)
          b. Loading time would be reduced greatly. (Not Sure!! Test still pending)

In [0]:
def loadGloveBase(gloveFile):
    print("Loading Glove Base")
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print ("Done.",len(model)," words loaded!")
    return model

In [0]:
glove_base = loadGloveBase('glove.42B.300d.txt')

Loading Glove Base
Done. 1917494  words loaded!


In [0]:
print(.0008*80000/(60))

1.0666666666666667


In [0]:
!pip install subprocess.run

  Stored in directory: /root/.cache/pip/wheels/55/ef/8b/a3dda73c54b3c1439af78f7a54d6035ad96c6e49432327cffd
Successfully built subprocess.run


In [0]:
import hashlib
import time
import gc

def myhash(input):
  hash_object = hashlib.md5(input.encode())
  output = hash_object.hexdigest()
  return output
  
def pickleGloveBase(gloveFile):
    print("\nPickling Glove Base\n")
    f = open(gloveFile,'r')
    
    total = 1917494
    
    dir_ = 'glove_42B_300d_embeddings/'
    
    model = {}
    
    i = 1
    del_t = 0
    for line in f:
        
        # Stopwtch Start
        start = time.time()
        
        # Reading (not b/w) the lines (rather themselves)
        splitLine = line.split()
        
        # Extracting Info
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
              
        # Hashing Words
        save = myhash(word)
                
        # Pickling word Embedding
        pickleOut(save, embedding)
        
        # Popen moving save to dir_
        subprocess.Popen(['mv', save, dir_])
        
        # Listing the saves
        model[word] = save
        
        # Checking the Progress 
        if np.remainder(i,20000) == 0:
          print(i, "words pickled, yet! ")
          print("  ETA:")
          show_time(ave_t*(total - i))
          print("\n")
          gc.collect()
          
        i += 1
        
        # Stopwatch Ends
        end = time.time()
        
        del_t = del_t + end - start 
        
        ave_t = del_t/i
        
        
    print ("Done.",len(model)," words pickled!")
    
    save_dict = model
    pickleOut('save_hash_dictionary', save_dict)
    
    subprocess.Popen(['mv', 'save_hash_dictionary', dir_])
    
    return save_dict
  
def access_glove_emb(word, save_dict):
  
  file = save_dict[word]
  emb = pickleIn(file)
  
  return emb

def show_time(amount):
  amount_dict = {1:'seconds',2:'mins',3:'hours',4:'days',5:'weeks',6:'months',7:'years'}
  count = 1

  factors = [60, 60, 24, 7, 4.3, 12]
  
  if amount < 60:
    print(amount, 'seconds')
  
  else:

    while 1:

      if count <= 6:
        amount = amount/factors[count-1]
      else:
        amount = amount/factors[5]

      if amount < 1:
        if count <= 7:
          print('    ',amount_, amount_dict[count])
        else:
          print('    ',amount_, amount_dict[7])
        break

      amount_ = amount
      count += 1


In [0]:
glove_save_dict = pickleGloveBase('glove.42B.300d.txt')


Pickling Glove Base

20000 words pickled, yet! 
  ETA:
     2.826790155585765 hours


40000 words pickled, yet! 
  ETA:
     2.955941277704345 hours


60000 words pickled, yet! 
  ETA:
     2.9386620213221026 hours


80000 words pickled, yet! 
  ETA:
     2.927246419706926 hours


100000 words pickled, yet! 
  ETA:
     2.9243342890147925 hours


120000 words pickled, yet! 
  ETA:
     2.87605182655345 hours


140000 words pickled, yet! 
  ETA:
     2.9352272042634997 hours


160000 words pickled, yet! 
  ETA:
     3.0217543693285007 hours


180000 words pickled, yet! 
  ETA:
     3.089802458701576 hours


200000 words pickled, yet! 
  ETA:
     3.088732645847225 hours


220000 words pickled, yet! 
  ETA:
     3.053544663995598 hours




KeyboardInterrupt: ignored

In [0]:
import subprocess

dir_ = 'glove_42B_300d_embeddings/'  
subprocess.Popen(['zip', '-r', 'glove_42B_300d_embeddings.zip', dir_])

In [0]:
!ls -l

total 6754464
-rw-r--r-- 1 root root       2559 Apr  6 12:29 081b16bcd52a14c866fe2ab1ae3c065a
drwxr-xr-x 2 root root   13713408 Apr  6 12:29 glove_42B_300d_embeddings
-rw-rw-r-- 1 root root 5025028820 Oct 24  2015 glove.42B.300d.txt
-rw-r--r-- 1 root root 1877800501 Oct 25  2015 glove.42B.300d.zip
drwxr-xr-x 1 root root       4096 Apr  4 20:20 sample_data


In [0]:
!rm -r glove_42B_300d_embeddings
!mkdir glove_42B_300d_embeddings
!rm 081b16bcd52a14c866fe2ab1ae3c065a

In [0]:
!ls glove_42B_300d_embeddings/

00003e3b9e5336685200ae85d21b4f5e  8036d66af8e98e892dc5191d7ef335fc
00008e40c5041a6a01ff63aec68b0978  8037ccea85006fee74f58f86edf1788e
000099656c16f8e84f50d10a65055c3d  8038115e9fd1f4d59751b0df520fb7b4
0000b7e3e7a2c5e0c705162c4cff6aff  80382dac961e832159cc3488e6d9f002
0000c2b100deee3441305e6c09860a08  80384bb51273c4bd610693564d31e860
00010818429f4ecb85fac9d49274838d  80388a0814e165797a2742ed5e3f7c0e
00018d45f7dce4da367d2dc7c8c78a8a  8038da89e49ac5eabb489cfc6cea9fc1
0001a68b9ec9d6277ef8eb3dabb40f0a  8038dae26b66e25427c290652c963c3d
0001bd723547963e9bc72f3b8a976e50  8039214c11ce594ea3961e04dd90c1d2
0001d89c5898ecde08fe4f73c73b673e  8039227006b7ce850a1cb99be5471e50
0001da15222a66ac4ce0061630434610  80393af8b3d99736c8b0d49d9a9da4ff
00022960a19b7e15e511ad578d4e61c2  8039777f0eeb78d275013d2561ad6c90
00023d2d36c58302fc4dc6c37b650d56  8039b4e0e6fe78bee7d3e5cfbfa343f0
00028107963a6d54284d80e54ce4dcd3  803a2f61bc314044dc33802270845de5
0002da9a6db278b9869c74801ce278fa  803a5613569e4b1ea435be42f113

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Scraping Text from Websites

In [0]:
!pip install nltk
!pip install beautifulsoup4
!pip install pytube

In [0]:
# To access the Websites
from urllib.request import urlopen, Request

# To get something meaningful out of those websites
import bs4
from bs4 import BeautifulSoup as sp

# For Preprocessing the Raw Extract from the Websites
import nltk

In [0]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}

reg_url = 'https://medium.com/gaplabs-engineering/how-we-failed-at-web-scraping-5db371757352'

req = Request(url=reg_url, headers=headers) 
html = urlopen(req).read() 

print(html)

b'<!DOCTYPE html><html xmlns:cc="http://creativecommons.org/ns#"><head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# medium-com: http://ogp.me/ns/fb/medium-com#"><meta http-equiv="Content-Type" content="text/html; charset=utf-8"><meta name="viewport" content="width=device-width, initial-scale=1.0, viewport-fit=contain"><title>How we failed at web scraping \xe2\x80\x93 GAPLabs Engineering \xe2\x80\x93 Medium</title><link rel="canonical" href="https://medium.com/gaplabs-engineering/how-we-failed-at-web-scraping-5db371757352"><meta name="title" content="How we failed at web scraping \xe2\x80\x93 GAPLabs Engineering \xe2\x80\x93 Medium"><meta name="referrer" content="unsafe-url"><meta name="description" content="Last week we held an intra-office hackathon as part of our monthly Team Day at GAPLabs. We were to build something that can help the company or its employees. Most of the ideas my team came up with\xe2\x80\xa6"><meta name="theme-color" content="#000000"><meta property="og:

In [0]:
soup = sp(html, 'html.parser')
web_text = soup.text

In [0]:
def get_text(url):
  headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}

  reg_url = 'https://medium.com/gaplabs-engineering/how-we-failed-at-web-scraping-5db371757352'

  req = Request(url=reg_url, headers=headers) 
  html = urlopen(req).read()
  
  soup = sp(html, 'html.parser')
  web_text = soup.text
  
  

In [0]:
# Mere COPY PASTE

'''
some_ = yo.split(' ')

some_new = []
for i in range(len(some_)):
    if len(some_[i]) < 25:
        some_new.append(some_[i])
        print(some_[i])
        
some_new_ = ' '.join(some_new)

some_new_new = some_new_.split('\n')

some_new_new_ = ' '.join(some_new_new)

print(some_new_new_)

bom = nltk.tokenize.treebank.TreebankWordTokenizer()
boo = bom.tokenize(some_new_new_)

'''

In [0]:
class WebSite:
  
  def _init_(self, url):
    self.url = url
    
  def get_text(site_object):
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}

    reg_url = site_object.url

    req = Request(url=reg_url, headers=headers) 
    html = urlopen(req).read()

    soup = sp(html, 'html.parser')
    web_text = soup.text
    
    
    

## Tokenization

In [0]:
import nltk
from nltk.tokenize.treebank import TreebankWordTokenizer

tokens = [token.lower() for token in tokens]